# Instalações
Conector do MySQL pela GCP

In [13]:
# install dependencies
import sys
!{sys.executable} -m pip install cloud-sql-python-connector["pymysql"] SQLAlchemy==2.0.7

## Autenticando na GCP
Para acessar por dentro do google o MySQL vamos autenticar usando o auth do colab/google.
Vai pedir permissão e talvez tenha que logar de novo.

In [14]:
from google.colab import auth
auth.authenticate_user()

## Parametros Produção
Na esquerda tem um vault, ali deve ser defina as variaves com dados sensiveis:


*   USER_MYSQL_PRD
*   PASSWORD_MYSQL_PRD
*   USER_MYSQL_DEV
*   PASSWORD_MYSQL_PRD

Vamos executar produção primeiro, pois no automatico a celula de dev vai substituir os valores.

In [15]:
#produção
from google.colab import userdata# isso é o vault do notebook, esquerda
DB_USER = userdata.get('USER_MYSQL_PRD')
DB_PASS = userdata.get('PASSWORD_MYSQL_PRD')
DB_NAME = "lmbr_prd_core"

project_id = "lmbr-instala-prd-brlm"
region = "southamerica-east1"
instance_name = "lmbr-instala-prd"

## Parametros Dev
Execução default, para evitar acidentes.

In [16]:
#dev
from google.colab import userdata# isso é o vault do notebook, esquerda
DB_USER = userdata.get('USER_MYSQL_DEV')
DB_PASS = userdata.get('PASSWORD_MYSQL_DEV')
DB_NAME = "lmbr_dev_core"

project_id = "lmbr-instala-dev-brlm"
region = "southamerica-east1"
instance_name = "lmbr-instala-dev-new"

In [17]:
# initialize parameters

INSTANCE_CONNECTION_NAME = f"{project_id}:{region}:{instance_name}" # i.e demo-project:us-central1:demo-instance
print(f"Your instance connection name is: {INSTANCE_CONNECTION_NAME}")

Your instance connection name is: lmbr-instala-dev-brlm:southamerica-east1:lmbr-instala-dev-new


## Configurando conexao
[Copiei daqui](https://colab.research.google.com/github/GoogleCloudPlatform/cloud-sql-python-connector/blob/main/samples/notebooks/mysql_python_connector.ipynb)


In [18]:

from google.cloud.sql.connector import Connector
import sqlalchemy


# initialize Connector object
connector = Connector()

# function to return the database connection object
def getconn():
    conn = connector.connect(
        INSTANCE_CONNECTION_NAME,
        "pymysql",
        user=DB_USER,
        password=DB_PASS,
        db=DB_NAME
    )
    return conn

# create connection pool with 'creator' argument to our connection object function
pool = sqlalchemy.create_engine(
    "mysql+pymysql://",
    creator=getconn,
)

## Funções
Vamos criar a update e select, para executar as operações de forçar update dos itens e select para percorrer a base de dados.

In [19]:
import pandas as pd

def update(data):
  oss = ",".join(map(str, data))
  print("Updating ", oss)
  # connect to connection pool
  with pool.connect() as db_conn:
    db_conn.execute(sqlalchemy.text("update servc_ord_evnt set priv_last_updt = now() where  SERVC_ORD_EVNT_SEQ in (" + str(oss) + ")"))
    db_conn.commit()
  return data

def select(sqlStr):
  # connect to connection pool
  with pool.connect() as db_conn:
    # query and fetch ratings table
    results = db_conn.execute(sqlalchemy.text(sqlStr)).fetchall()

    return pd.DataFrame(results)

# start last with min
#lastSequence = select("select min(SERVC_ORD_EVNT_SEQ) as mn from servc_ord_evnt where SERVC_ORD_EVNT_STUS_IND = '72'").iloc[0,0]
#print("Menor evento 72", lastSequence)
#lastSequence = lastSequence - 1
updated = []
lastSequence = select("select min(SERVC_ORD_EVNT_SEQ) as mn from servc_ord_evnt where SERVC_ORD_EVNT_STUS_IND = '72'").iloc[0,0]


### Arrumador
Isso vai percorrer a tabela de 5000 por vez
Se tiver eventos 72 ele manda atualizar
No fim ele pega o maior id de evento para buscar a proxima janela.

In [20]:
size = 1

while size > 0:
  janela = select("""
  select SERVC_ORD_EVNT_SEQ,SERVC_ORD_EVNT_STUS_IND,priv_last_updt
  from servc_ord_evnt
  WHERE SERVC_ORD_EVNT_SEQ > """ + str(lastSequence) + """
  order by SERVC_ORD_EVNT_SEQ limit 5000
  """)
  size = len(janela)
  toUpdate = []
  for i, v in janela.iterrows():
    if v['SERVC_ORD_EVNT_STUS_IND'] == "72":
      toUpdate.append(v['SERVC_ORD_EVNT_SEQ'])
    lastSequence = v['SERVC_ORD_EVNT_SEQ']
  # dont call update when empty
  if len(toUpdate) > 0:
    updated = updated + update(toUpdate)


print(lastSequence)
print("Updated", len(updated))

Updating  3849014
Updating  3854517,3854718,3855475,3855485
Updating  3856100,3857407,3857409,3857716,3860366,3860367,3860590,3860664
Updating  3861133,3861153,3861244,3861549,3861568,3861684,3861685,3861706,3861707,3861732,3861785,3861817,3861849,3861872,3861927,3861953,3861983,3862029,3862247,3862304,3862362,3862469,3862762,3862793,3862817,3862931,3862964,3862965,3862967,3862968,3862969,3862970,3862987,3862989,3863048,3863081,3863118,3863139,3863142,3863167,3863195,3863311,3863546,3863561,3863562,3863563,3863564,3863565,3863568,3863570,3863573,3863598,3863725,3863756,3863794,3863817,3863836,3863864,3863951,3863970,3863971,3864003,3864027,3864054,3864118,3864139,3864185,3864276,3864362,3864406,3864437,3864475,3864536,3864537,3864538,3864649,3864676,3864694,3864731,3864753,3864768,3866270,3866298
Updating  3867320,3868116,3869854
Updating  3873680
Updating  3879122,3880500,3880598,3882037,3882211,3882481
Updating  3883571,3883629,3883698,3884119,3884152,3884172,3884189,3884211,3884236,

In [11]:
print(lastSequence) #42004949
print("Updated", len(updated)) #255821 de 42002844


4361943
Updated 845


## Extração
O codigo salva em um arquivo a lista transformada em dataframe.
Depois força o download do arquivo para a maquina local.

In [12]:
from google.colab import files
updatedDF = pd.DataFrame(updated)
updatedDF.to_csv("updated2.csv")
files.download('updated2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import json

# a Python object (dict):
x = {
  "name": "John",
  "age": 30,
  "city": "New York"
}

# convert into JSON:
y = json.dumps(x)

# the result is a JSON string:
print(y)

{"name": "John", "age": 30, "city": "New York"}


## Conferencia
//TODO conectar pela GCP